In [1]:
import os
import json
from collections import defaultdict
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string
from tqdm import tqdm
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
DATA_FOLDER = "/content/drive/MyDrive/AI4E/Project_ML NLP/data"

In [4]:
hot_news = []

# Trang web vnexpress

In [ ]:
VNEXPRESS_DATA_FOLDER = os.path.join(DATA_FOLDER, "vnexpress")
os.listdir(VNEXPRESS_DATA_FOLDER)

['thời_sự.json',
 'thế_giới.json',
 'kinh_doanh.json',
 'bất_động_sản.json',
 'khoa_học.json',
 'giải_trí.json',
 'thể_thao.json',
 'pháp_luật.json',
 'giáo_dục.json',
 'sức_khỏe.json',
 'đời_sống.json',
 'du_lịch.json',
 'số_hóa.json',
 'xe.json',
 'hot_news.json']

In [ ]:
# Gộp các thể loại file và gộp thành 1 list
all_docs = []
for file_name in os.listdir(VNEXPRESS_DATA_FOLDER):
    with open(os.path.join(VNEXPRESS_DATA_FOLDER, file_name)) as fIn:
        current_docs = json.load(fIn)
    # web_news field
    for doc in current_docs:
        doc["web_news"] = "vnexpress"
        if file_name == 'hot_news.json':
          doc['hot_news'] = True
    all_docs.extend(current_docs)
len(all_docs)

5713

In [ ]:
len([doc for doc in all_docs if 'hot_news' in doc])

17

In [ ]:
# Kiểm tra một văn bản
all_docs[0]

{'url': 'https://vnexpress.net/dai-vtc-ky-niem-20-nam-thanh-lap-4780701.html',
 'title': 'Đài VTC kỷ niệm 20 năm thành lập',
 'description': 'Từ 50 nhân sự đầu tiên, "cánh sóng" VTC đã vươn khắp đất nước với 15 kênh và một báo điện tử, đáp ứng nhu cầu của hàng triệu khán giả.',
 'content': 'Ngày 12/8, tại Hà Nội, Đài Truyền hình kỹ thuật số VTC tổ chức kỷ niệm 20 năm thành lập. Ban Biên tập Truyền hình Kỹ thuật số VTC - tiền thân của Đài VTC - được ra mắt ngày 19/8/2004. Với 50 nhân sự ban đầu, chương trình truyền hình đầu tiên được lên sóng sau hai tuần.\n"Truyền hình số VTC ra đời gây tiếng vang vì hình rõ nét, tiếng trung thực, truyền hình tốt và xa, ưu điểm lớn nhất là lưu trữ được tín hiệu", nguyên Bộ trưởng Bưu chính Viễn thông Đỗ Trung Tá nhận xét.\nÔng Lưu Vũ Hải, nguyên Phó tổng giám đốc Đài VTC, sau này có thời gian là Cục trưởng Phát thanh Truyền hình (Bộ Thông tin và Truyền thông) từng nói: "VTC ra đời được xã hội đón nhận nồng nhiệt và từng bước thuyết phục các cơ quan chứ

## Xử lý description

In [ ]:
# Check description
all_descriptions = []
for doc in all_docs:
    all_descriptions.append(doc["description"])

Kiểm tra vài samples

In [ ]:
all_descriptions[:10]

# Một vài description bị dính địa điểm ở trước

['Từ 50 nhân sự đầu tiên, "cánh sóng" VTC đã vươn khắp đất nước với 15 kênh và một báo điện tử, đáp ứng nhu cầu của hàng triệu khán giả.',
 'ĐỒNG NAIÔtô bán tải chở ba người đi qua đoạn giao cắt đường sắt khi rào chắn đã đóng bị tàu tông văng gần 10 m, hai người tử vong, tối 28/7.',
 'BÌNH DƯƠNGVòng xoay Định Hòa trên đường huyết mạch Mỹ Phước - Tân Vạn thường xảy ra lật xe sẽ được đập bỏ, thay bằng đèn tín hiệu kết hợp tiểu đảo.',
 'HĐND tỉnh Quảng Ninh thông qua Nghị quyết miễn nhiệm chức Chủ tịch HĐND và cho thôi nhiệm vụ đại biểu HĐND tỉnh nhiệm kỳ 2021-2026 với ông Nguyễn Xuân Ký.',
 'Cuộc đàm phán đa phương đầu tiên với các cường quốc ở Geneve để lại cho Việt Nam nhiều bài học đắt giá, theo nguyên Phó viện trưởng Viện Lịch sử quân sự Việt Nam Trần Ngọc Long.',
 'Sau nhiều vụ cây đổ, tét nhánh, công ty Công viên cây xanh TP HCM đề xuất sử dụng flycam để rà soát tình trạng, thuê xe loại 40 m xử lý khiếm khuyết trên cây.',
 'TRUNG QUỐCTổng Bí thư, Chủ tịch nước Tô Lâm dâng hương tưở

In [ ]:
def remove_location_description(description_str):
    # Calculate the maximum isupper or isspace first
    count = 0
    is_last_space = False # Only cut when the last is not space
    for chac in description_str:
      # '-': THỪA THIÊN HUẾ
        if chac.isupper() or chac.isspace() or chac == '-':
            count += 1
            if chac.isupper():
                is_last_space = False
            else:
                is_last_space = True
        else:
            if count >= 1: # Remain the first capitalized (or not word) character
                count -= 1
            break

    return description_str[count:] if not is_last_space else description_str

In [ ]:
all_descriptions = []
for doc in all_docs:
    doc["description"] = remove_location_description(doc["description"])
    all_descriptions.append(doc["description"])

In [ ]:
all_descriptions[:10]

['Từ 50 nhân sự đầu tiên, "cánh sóng" VTC đã vươn khắp đất nước với 15 kênh và một báo điện tử, đáp ứng nhu cầu của hàng triệu khán giả.',
 'Ôtô bán tải chở ba người đi qua đoạn giao cắt đường sắt khi rào chắn đã đóng bị tàu tông văng gần 10 m, hai người tử vong, tối 28/7.',
 'Vòng xoay Định Hòa trên đường huyết mạch Mỹ Phước - Tân Vạn thường xảy ra lật xe sẽ được đập bỏ, thay bằng đèn tín hiệu kết hợp tiểu đảo.',
 'HĐND tỉnh Quảng Ninh thông qua Nghị quyết miễn nhiệm chức Chủ tịch HĐND và cho thôi nhiệm vụ đại biểu HĐND tỉnh nhiệm kỳ 2021-2026 với ông Nguyễn Xuân Ký.',
 'Cuộc đàm phán đa phương đầu tiên với các cường quốc ở Geneve để lại cho Việt Nam nhiều bài học đắt giá, theo nguyên Phó viện trưởng Viện Lịch sử quân sự Việt Nam Trần Ngọc Long.',
 'Sau nhiều vụ cây đổ, tét nhánh, công ty Công viên cây xanh TP HCM đề xuất sử dụng flycam để rà soát tình trạng, thuê xe loại 40 m xử lý khiếm khuyết trên cây.',
 'Tổng Bí thư, Chủ tịch nước Tô Lâm dâng hương tưởng niệm liệt sĩ Phạm Hồng Th

Kiểm tra bằng độ dài

In [ ]:
max([len(desc) for desc in all_descriptions])

240

In [ ]:
min([len(desc) for desc in all_descriptions])

67

## Xử lý author

In [ ]:
all_authors = []
for doc in all_docs:
    all_authors.append(doc["metadata"]["author"])

all_authors[:10]

['Xuân Hoa',
 'Phước Tuấn',
 'Phước Tuấn',
 'Lê Tân',
 'Phạm Dự - Hoàng Phương',
 'Gia Minh - Đình Văn',
 'Ngọc Ánh (Theo TTXVN/Báo Thế giới và Việt Nam)',
 'Võ Thạnh',
 'Hoàng Phương - Gia Chính',
 'Xuân Hoa']

In [ ]:
# Kiểm tra tác giả có độ dài >= 50
for doc in all_docs:
    if len(doc["metadata"]["author"]) >= 50:
        print(doc["metadata"]["author"])

# parse errors, --> split with the first \n

*Lời tưởng niệm Tổng Bí thư Nguyễn Phú Trọng
Lễ thượng cờ rủ Quốc tang Tổng Bí thư Nguyễn Phú Trọng
6h ngày 25/7, cờ rủ tại quảng trường Ba Đình được kéo lên, cả nước bắt đầu hai ngày Quốc tang Tổng Bí thư Nguyễn Phú Trọng. 24
Người dân lên chùa tưởng nhớ Tổng Bí thư Nguyễn Phú Trọng
Nhiều người dân TP HCM, Long An đến chùa để thắp hương, tỏ lòng thành kính Tổng Bí thư Nguyễn Phú Trọng, ngày 23/7. 64
Nhiều nơi treo cờ rủ trước Quốc tang Tổng bí thư Nguyễn Phú Trọng
Nhiều cơ quan, tổ chức, nhà dân ở Hà Nội, Huế, TP HCM treo cờ rủ tưởng nhớ Tổng bí thư Nguyễn Phú Trọng trước khi Quốc tang diễn ra vào 25-26/7. 32
Giang Huy - Thanh Tùng
Dòng người xếp hàng xuyên trưa chờ viếng Tổng Bí thư tại quê nhà
Người dân xếp hàng dài 2 km từ cây bồ đề ngoài cổng làng đến điểm viếng ở nhà văn hóa thôn Lại Đà để chờ viếng Tổng Bí thư Nguyễn Phú Trọng tại quê nhà xã Đông Hội, huyện Đông Anh, trưa 25/7. 36
Lãnh đạo Đảng, Nhà nước viếng Tổng Bí thư Nguyễn Phú Trọng
Chủ tịch nước Tô Lâm, Thủ tướng Phạm 

In [ ]:
# Xử lý split
for doc in all_docs:
    len_author = len(doc["metadata"]["author"])
    if len_author >= 40 and len_author < 100:
        doc["metadata"]["author"] = doc["metadata"]["author"].split("\n")[0].strip()
    if len_author >= 100 or len_author == 0:
        doc["metadata"]["author"] = "Unknown"
    # Kiểm tra và loại bỏ các tác giả có chứa URL: 'Mời độc giả gửi bài, câu hỏi tại đây hoặc về dulich@vnexpress.net'
    if 'Mời độc giả gửi bài, câu hỏi tại đây' in doc["metadata"]["author"]:
        doc["metadata"]["author"] = "Unknown"

In [ ]:
all_authors = []
for doc in all_docs:
    all_authors.append(doc["metadata"]["author"])

all_authors[:30]

['Xuân Hoa',
 'Phước Tuấn',
 'Phước Tuấn',
 'Lê Tân',
 'Phạm Dự - Hoàng Phương',
 'Gia Minh - Đình Văn',
 'Ngọc Ánh (Theo TTXVN/Báo Thế giới và Việt Nam)',
 'Võ Thạnh',
 'Hoàng Phương - Gia Chính',
 'Xuân Hoa',
 'Quỳnh Trần',
 'Hoàng Phương',
 'Gia Chính',
 'Unknown',
 'Gia Chính',
 'Lê Tuyết',
 'Sơn Hà',
 'Ngọc Ánh',
 'Trần Hóa',
 'Sơn Hà',
 'Việt Quốc',
 'Hồng Chiêu',
 'Ngọc Trường - Hồng Chiêu',
 'Unknown',
 'Đức Hùng',
 'Đức Hùng',
 'Hồng Chiêu',
 'Phạm Dự - Lê Tuyết',
 'Unknown',
 'Gia Minh']

## Xử lý contents

In [ ]:
contents = []
for doc in all_docs:
    contents.append(doc["content"])

Kiểm tra độ dài contents

In [ ]:
len_contents = [len(content) for content in contents]

In [ ]:
min(len_contents), max(len_contents)

(0, 29147)

In [ ]:
# Kiểm tra tại sao lại có docs len == 0
for doc in all_docs:
    if len(doc["content"]) == 0:
        print(doc)

# --> Mấy MCQs báo, thông báo, video....

{'url': 'https://vnexpress.net/ong-duong-trung-quoc-long-dan-la-thuoc-do-chuan-muc-nhat-4774273.html', 'title': "Ông Dương Trung Quốc: 'Lòng dân là thước đo chuẩn mực nhất'", 'description': 'Viếng Tổng Bí thư Nguyễn Phú Trọng tại Nhà tang lễ quốc gia sáng 26/7, nhà sử học Dương Trung Quốc nói "lòng dân là thước đo công bằng và chuẩn mực nhất, bà con khắp nơi đến đây đã thể hiện rất rõ điều đó".', 'content': '', 'metadata': {'cat': 'Thời sự', 'subcat': 'Chính trị', 'published_date': '2024-07-26T07:00:00+07:00', 'author': 'Unknown'}, 'web_news': 'vnexpress'}
{'url': 'https://vnexpress.net/truc-tiep-le-vieng-tong-bi-thu-nguyen-phu-trong-4774121.html', 'title': 'Người dân tiễn biệt Tổng Bí thư Nguyễn Phú Trọng', 'description': 'Tối 25/7, hàng nghìn người dân mặc trang phục đen xếp hàng vào bái biệt Tổng Bí thư Nguyễn Phú Trọng ở Nhà tang lễ quốc gia số 5 Trần Thánh Tông, Hà Nội, nhiều người bật khóc.', 'content': '', 'metadata': {'cat': 'Thời sự', 'subcat': 'Chính trị', 'published_date': '

In [ ]:
## -> remove all < 300 characters
all_docs_news = []
for doc in all_docs:
    if len(doc["content"]) >= 300:
        all_docs_news.append(doc)
all_docs = all_docs_news
len(all_docs)

5438

## Xử lý publish date

Đưa publish date về dạng chuẩn timestamp

In [ ]:
from datetime import datetime
def convert2timestamp(date_string):
    dt = datetime.fromisoformat(date_string)
    return int(dt.timestamp())

for doc in all_docs:
    doc['metadata']['published_date'] = convert2timestamp(doc['metadata']['published_date'])

## Tách hot news

Tách hot news ra để khi tổng hợp 3 trang báo lại, shuffle rồi mới thêm hot_news để hot_news ở cuối cùng

In [ ]:
hot_news.extend([doc for doc in all_docs if 'hot_news' in doc])
len(hot_news)

16

In [ ]:
for doc in hot_news:
  all_docs.remove(doc)

5422

## Lưu processed data

In [ ]:
with open(os.path.join(DATA_FOLDER, "vnexpress_processed.json"), "w") as fOut:
    json.dump(all_docs, fOut, indent=4, ensure_ascii=False)

# Trang web dantri

Similar to trang web vnexpress

In [ ]:
DANTRI_DATA_FOLDER = os.path.join(DATA_FOLDER, "dantri")
files = os.listdir(DANTRI_DATA_FOLDER)
files, len(files)

(['thế-giới.json',
  'kinh-doanh.json',
  'bất-động-sản.json',
  'thể-thao.json',
  'việc-làm.json',
  'nhân-ái.json',
  'pháp-luật.json',
  'du-lịch.json',
  'đời-sống.json',
  'tình-yêu.json',
  'khoa-học---công-nghệ.json',
  'xã-hội.json',
  'sức-mạnh-số1.json',
  'an-sinh.json',
  'giáo-dục.json',
  'giải-trí.json',
  'xe-++.json',
  'sức-khỏe.json'],
 18)

In [ ]:
# Gộp các thể loại file và gộp thành 1 list
all_docs = []
for file_name in os.listdir(DANTRI_DATA_FOLDER):
    with open(os.path.join(DANTRI_DATA_FOLDER, file_name)) as fIn:
        current_docs = json.load(fIn)
    for doc in current_docs:
        doc["web_news"] = "dantri"
    all_docs.extend(current_docs)
len(all_docs)

5960

## Xử lý description

In [ ]:
all_descriptions = []
for doc in all_docs:
    all_descriptions.append(doc['description'])

In [ ]:
all_descriptions[:50]

['Thủ tướng Tây Ban Nha Pedro Sánchez tuyên bố thực hiện hành động pháp lý để "ngăn chặn ngành tư pháp bị lợi dụng cho động cơ chính trị".',
 'Chương trình vui đón Tết Trung thu không chỉ là dịp để các gia đình người Việt tại Frankfurt, Đức hội ngộ, mà còn là cơ hội để các em nhỏ tìm hiểu, trải nghiệm nét văn hóa truyền thống của dân tộc.',
 'Kiev tuyên bố đã dụ hàng thành công một binh sĩ Nga, người sau đó đã ném lựu đạn vào sở chỉ huy rồi đào tẩu sang khu vực do Ukraine kiểm soát.',
 'Quan chức Nga tuyên bố Moscow hiện chưa sẵn sàng đàm phán hòa bình với Ukraine sau cuộc tấn công của Kiev vào vùng Kursk.',
 'Giới chức Nhật Bản bàn bạc về phương án nhằm thay đổi luật kế vị trong bối cảnh Hoàng gia nước này ngày càng ít thành viên.',
 'Từ khắp thế giới, các bạn nhỏ người Việt và gốc Việt hướng về quê hương thông qua hàng loạt hoạt động đầy ý nghĩa đóng góp vào chương trình kêu gọi 10.000 liều vắc xin cho Việt Nam.',
 'Triều Tiên lên án kế hoạch của Mỹ bán trực thăng Apache cho Hàn Quốc

In [ ]:
max([len(desc) for desc in all_descriptions])

310

In [ ]:
min([len(desc) for desc in all_descriptions])

57

In [ ]:
len(all_docs)

5960

## Xử lý author

In [ ]:
all_authors = []
for doc in all_docs:
    all_authors.append(doc['metadata']['author'])
all_authors[:20]

['Đức Hoàng',
 'CTV',
 'Đức Hoàng',
 'Thành Đạt',
 'Đức Hoàng',
 'Đức Hoàng',
 'Thành Đạt',
 'Minh Phương',
 'Minh Phương',
 'Minh Phương',
 'Đức Hoàng',
 'Thanh Thành',
 'Đức Hoàng',
 'Minh Phương',
 'Minh Phương',
 'Thành Đạt',
 'Đức Hoàng',
 'Đức Hoàng',
 'Minh Phượng',
 'Đức Hoàng']

In [ ]:
for author in all_authors:
    if(len(author) > 50):
        print(author)

Trích sách "Hạt giống tâm hồn 4 và Từ những điều bình dị"


In [ ]:
for doc in all_docs:
  if(len(doc['metadata']['author']) > 50):
    doc['metadata']['author'] = "Unknown"

## Xử lý contents

In [ ]:
contents = []
for doc in all_docs:
    contents.append(doc['content'])

In [ ]:
max([len(content) for content in contents])

19177

In [ ]:
min([len(content) for content in contents])

0

In [ ]:
len([doc for doc in all_docs if doc['metadata']['cat'] == 'AN SINH'])

360

In [ ]:
dic = defaultdict(int)
for doc in all_docs:
    if(len(doc['content']) <= 300 or len(doc['description']) <= 50):
      dic[doc['metadata']['cat']] += 1
      # print(json.dumps(doc, indent=4, ensure_ascii=False))
dic

defaultdict(int,
            {'THẾ GIỚI': 1,
             'XÃ HỘI': 1,
             'BẤT ĐỘNG SẢN': 2,
             'LAO ĐỘNG - VIỆC LÀM': 3,
             'PHÁP LUẬT': 2,
             'DU LỊCH': 2,
             'ĐỜI SỐNG': 3,
             'KHOA HỌC - CÔNG NGHỆ': 6,
             'SỨC KHỎE': 3})

In [ ]:
all_new_docs = []
for doc in all_docs:
  if(len(doc['content']) >= 300):
    all_new_docs.append(doc)
all_docs = all_new_docs
len(all_docs)

5937

## Xử lý publish date

In [ ]:
from datetime import datetime
def convert2timestamp(date_string):
    dt = datetime.fromisoformat(date_string)
    return int(dt.timestamp())
for doc in all_docs:
    doc['metadata']['published_date'] = convert2timestamp(doc['metadata']['published_date'])

In [ ]:
all_docs[0]['metadata']['published_date']

1722397907

## Lưu processed data

In [ ]:
with open(os.path.join(DATA_FOLDER, "dantri_processed.json"), "w") as fOut:
    json.dump(all_docs, fOut, indent=4, ensure_ascii=False)

# Trang web vietnamnet

In [ ]:
VIETNAMNET_DATA_FOLDER = os.path.join(DATA_FOLDER, "vietnamnet")
files = os.listdir(VIETNAMNET_DATA_FOLDER)
files

['chính_trị.json',
 'kinh_doanh.json',
 'thể_thao.json',
 'thế_giới.json',
 'giáo_dục.json',
 'giải_trí.json',
 'đời_sống.json',
 'sức_khỏe.json',
 'thông_tin_và_truyền_thông.json',
 'pháp_luật.json',
 'ô_tô_xe_máy.json',
 'bất_động_sản.json',
 'bạn_đọc.json',
 'tuần_việt_nam.json',
 'công_nghiệp_hỗ_trợ.json',
 'thị_trường_tiêu_dùng.json',
 'dân_tộc_-_tôn_giáo.json',
 'nông_thôn_mới.json',
 'dân_tộc_thiểu_số_và_miền_núi.json',
 'nội_dung_chuyên_đề.json',
 'bảo_vệ_người_tiêu_dùng.json',
 'du_lịch.json',
 'văn_hóa.json',
 'hot_news.json']

In [ ]:
# Group all the file
all_docs = []
for file_name in os.listdir(VIETNAMNET_DATA_FOLDER):
    with open(os.path.join(VIETNAMNET_DATA_FOLDER, file_name)) as fIn:
        current_docs = json.load(fIn)
    for doc in current_docs:
        doc["web_news"] = "vietnamnet"
        if file_name == 'hot_news.json':
          doc['hot_news'] = True
    all_docs.extend(current_docs)

In [ ]:
len([doc for doc in all_docs if 'hot_news' in doc])

27

## Xử lý description

In [ ]:
all_descriptions = []
for doc in all_docs:
  all_descriptions.append(doc['description'])

In [ ]:
all_descriptions[:20]

['Chánh án TAND Tối cao Nguyễn Hòa Bình cho rằng, nếu không tách vụ án có người chưa thành niên phạm tội mà để vụ án chung, xét xử công khai thì sẽ ảnh hưởng đến con đường hoàn lương, các cháu sẽ luôn bị ám ảnh bởi một tuổi thơ phạm tội.',
 'Sáng 8/8, ông Trần Thanh Mẫn, Chủ tịch Quốc hội, cùng đoàn công tác trung ương đã đến kiểm tra tiến độ xây dựng sân bay Long Thành (Đồng Nai).',
 'Sáng 2/8, Văn phòng Chủ tịch nước phối hợp với Bộ Công an, Bộ Ngoại giao, TAND tối cao tổ chức họp báo công bố quyết định về đặc xá năm 2024 của Chủ tịch nước.',
 'Sáng nay, Tổng Bí thư, Chủ tịch nước Tô Lâm và Phu nhân cùng đoàn đại biểu cấp cao Đảng, Nhà nước Việt Nam rời Hà Nội lên đường thăm cấp Nhà nước tới Trung Quốc.',
 'Tỉnh Yên Bái vừa tổ chức Hội nghị công bố các quyết định của Ban Thường vụ Tỉnh ủy, UBND tỉnh về công tác cán bộ đối với một số sở, ngành.',
 'Trong các ngày qua, lãnh đạo các đảng, các nước tiếp tục gửi thư, điện chúc mừng Tổng Bí thư, Chủ tịch nước Tô Lâm nhân dịp được Ban Chấp 

In [ ]:
max([len(desc) for desc in all_descriptions])

250

In [ ]:
min([len(desc) for desc in all_descriptions])

15

In [ ]:
for doc in all_docs:
  if(len(doc['description']) <= 50):
    print(json.dumps(doc, indent=4, ensure_ascii=False))

{
    "url": "https://vietnamnet.vn/khuon-vien-xanh-mat-ben-trong-5-co-so-dai-hoc-fpt-2306677.html",
    "title": "Khuôn viên xanh mát bên trong 5 cơ sở Đại học FPT",
    "description": "Khuôn viên xanh mát bên trong 5 cơ sở Đại học FPT",
    "content": "Trường Đại học FPT hiện diện tại 5 thành phố lớn trên cả nước: Hà Nội, Đà Nẵng, Quy Nhơn, TP.HCM và Cần Thơ, sinh viên có đặc quyền ‘Local Exchange’ - chuyển đôi địa điểm học giữa các cơ sở. \nTheo đó, ngoài địa chỉ đã đăng ký theo học, sinh viên có thể trải nghiệm ít nhất 1 học kỳ tại bất kỳ cơ sở nào trên cả nước.\nĐại học FPT Hà Nội với khu vườn xanh mát và hồ sinh thái - nơi thiên nhiên giao thoa với tri thức tại Khu công nghệ cao Hòa Lạc\nĐH FPT Hà Nội sở hữu khuôn viên 30ha với những lối đi rợp bóng cây, sức chứa giảng đường 25.000 sinh viên, sức chứa ký túc xá 7.000 sinh viên. Tòa nhà giảng đường được thiết kế với kiến trúc hiện đại, độc đáo, hài hòa với cảnh quan thiên nhiên xung quanh.\nHồ sen sinh thái tại trường là một điểm 

In [ ]:
new_all_docs = []
for doc in all_docs:
  if(len(doc['description']) > 50):
    new_all_docs.append(doc)
all_docs = new_all_docs
len(all_docs)

9884

## Xử lý author

In [ ]:
all_authors = []
for doc in all_docs:
    all_authors.append(doc['metadata']['author'])
all_authors[:20]

['Thu Hằng',
 'Hoàng Anh- ĐN',
 'Ông Ngọc cho biết thêm, năm 2022 có 16 phạm nhân quốc tịch nước ngoài được đặc xá, còn năm 2021 có 21 phạm nhân.',
 'Đại sứ Hùng Ba cho biết, phía Trung Quốc kỳ vọng và mong đợi chuyến thăm sẽ tiếp tục kế thừa tình hữu nghị Trung - Việt, củng cố tin cậy chính trị, thúc đẩy xây dựng Cộng đồng chia sẻ tương lai Trung Quốc - Việt Nam có ý nghĩa chiến lược, đóng góp tích cực vào hòa bình, ổn định và phát triển tại khu vực và thế giới.',
 'Đức Hoàng',
 'Trần Thường',
 'Thu Hằng',
 'Đại sứ Mỹ tại Việt Nam Marc Knapper và Đại sứ Nga tại Việt Nam Gennady Bezdetko ghi sổ tang tại Nhà khách Chính phủ.',
 'Thu Hằng',
 'Theo TTXVN',
 'Trần Thường',
 'Trần Thường',
 'Trần Tuyên',
 '“Vừa qua các nghị quyết của Quốc hội, Bộ Chính trị đã cho phép Hà Nội, TPHCM triển khai và có những đề án riêng. Tỷ lệ đóng góp của ngành này rất tích cực, lan tỏa, phát huy sức mạnh mềm của văn hóa, văn hóa giữ vai trò kiến tạo sự phát triển bền vững”, Bộ trưởng Nguyễn Văn Hùng nói.',
 '

In [ ]:
for doc in all_docs:
  if(doc['metadata']['author'] == '' or len(doc['metadata']['author']) >= 50):
    doc['metadata']['author'] = "Unknown"
  elif('\n' in doc['metadata']['author']):
    doc['metadata']['author'] = doc['metadata']['author'].split('\n')[0].strip()

In [ ]:
all_authors = []
for doc in all_docs:
    all_authors.append(doc['metadata']['author'])
all_authors[:20]

['Thu Hằng',
 'Hoàng Anh- ĐN',
 'Unknown',
 'Unknown',
 'Đức Hoàng',
 'Trần Thường',
 'Thu Hằng',
 'Unknown',
 'Thu Hằng',
 'Theo TTXVN',
 'Trần Thường',
 'Trần Thường',
 'Trần Tuyên',
 'Unknown',
 'Quang Phong',
 'Uknown',
 'Unknown',
 'Theo TTXVN',
 'Hồ Văn',
 'Trần Thường']

## Xử lý contents

In [ ]:
contents = []
for doc in all_docs:
    contents.append(doc['content'])

In [ ]:
contents[:5]

['Chiều ngày 13/8, Ủy ban Thường vụ Quốc hội cho ý kiến về một số vấn đề lớn trong giải trình, tiếp thu, chỉnh lý dự thảo Luật Tư pháp người chưa thành niên.\nMột trong những nội dung đáng chú ý được các đại biểu quan tâm là việc tách vụ án có người chưa thành niên phạm tội.\nTách vụ án là cần thiết\nChủ nhiệm Ủy ban Tư pháp Lê Thị Nga cho biết, quy định này vẫn có ý kiến khác nhau. Quá trình chỉnh lý, trên cơ sở nghiên cứu, thảo luận kỹ lưỡng các chính sách mới của dự thảo luật, đại diện các cơ quan tham gia chỉnh lý (Bộ Công an, Bộ Tư pháp, VKSND Tối cao, TAND Tối cao) đều thống nhất với dự thảo luật phải tách vụ án có người chưa thành niên phạm tội để giải quyết riêng.\n[Chủ nhiệm Ủy ban Tư pháp Lê Thị Nga. Ảnh: QH]\nTuy nhiên, bà Nga cho hay, thảo luận trong Thường trực Ủy ban Tư pháp thì có 2 loại ý kiến với vấn đề này.\nThứ nhất, tán thành quy định phải tách vụ án với người chưa thành niên để giải quyết riêng; đồng thời, đề nghị giao Bộ Công an, VKSND Tối cao, TAND Tối cao xây dự

In [ ]:
max([len(content) for content in contents])

48335

In [ ]:
min([len(content) for content in contents])

0

In [ ]:
all_docs_news = []
for doc in all_docs:
    if len(doc["content"]) >= 300:
        all_docs_news.append(doc)
all_docs = all_docs_news
len(all_docs)

9147

## Xử lý publish date

In [ ]:
def convert2timestamp(date_string):
    date_string = "".join(date_string.split())
    dt = datetime.fromisoformat(date_string)
    return int(dt.timestamp())

for doc in all_docs:
    doc['metadata']['published_date'] = convert2timestamp(doc['metadata']['published_date'])

In [ ]:
all_docs[0]['metadata']['published_date']

1723544460

## Tách Hot news

In [ ]:
hot_news.extend([doc for doc in all_docs if 'hot_news' in doc])
len(hot_news)

43

In [ ]:
for doc in hot_news:
  if doc in all_docs:
    all_docs.remove(doc)

In [ ]:
all_docs[0], len(all_docs)

({'url': 'https://vietnamnet.vn/ong-nguyen-hoa-binh-tach-an-de-cac-chau-khong-bi-am-anh-boi-tuoi-tho-pham-toi-2311387.html',
  'title': 'Ông Nguyễn Hòa Bình: Tách án để các cháu không bị ám ảnh bởi tuổi thơ phạm tội',
  'description': 'Chánh án TAND Tối cao Nguyễn Hòa Bình cho rằng, nếu không tách vụ án có người chưa thành niên phạm tội mà để vụ án chung, xét xử công khai thì sẽ ảnh hưởng đến con đường hoàn lương, các cháu sẽ luôn bị ám ảnh bởi một tuổi thơ phạm tội.',
  'content': 'Chiều ngày 13/8, Ủy ban Thường vụ Quốc hội cho ý kiến về một số vấn đề lớn trong giải trình, tiếp thu, chỉnh lý dự thảo Luật Tư pháp người chưa thành niên.\nMột trong những nội dung đáng chú ý được các đại biểu quan tâm là việc tách vụ án có người chưa thành niên phạm tội.\nTách vụ án là cần thiết\nChủ nhiệm Ủy ban Tư pháp Lê Thị Nga cho biết, quy định này vẫn có ý kiến khác nhau. Quá trình chỉnh lý, trên cơ sở nghiên cứu, thảo luận kỹ lưỡng các chính sách mới của dự thảo luật, đại diện các cơ quan tham gia

## Lưu processed data

In [ ]:
with open(os.path.join(DATA_FOLDER, "vietnamnet_processed.json"), "w", encoding='utf-8') as fOut:
    json.dump(all_docs, fOut, indent=4, ensure_ascii=False)

# Gộp dữ liệu từ 3 website vào thành một file

In [ ]:
# Đọc từng file một
all_json_files = ["dantri_processed.json", "vietnamnet_processed.json", "vnexpress_processed.json"]
all_docs = []

for file_name in all_json_files:
    with open(os.path.join(DATA_FOLDER, file_name)) as fOut:
        all_docs.extend(json.load(fOut))

len(all_docs)


20479

In [ ]:
import random
random.seed(73) # Shuffle cho đó
random.shuffle(all_docs)

In [ ]:
all_docs.extend(hot_news)
len(all_docs)

20522

In [ ]:
all_docs[-3:]

[{'url': 'https://vietnamnet.vn/tuyen-viet-nam-khi-ong-kim-sang-sik-mo-khoa-hang-cong-2316868.html',
  'title': 'Tuyển Việt Nam: Khi ông Kim Sang Sik mở khoá hàng công',
  'description': 'Tuyển Việt Nam còn rất nhiều vấn đề nếu muốn đòi lại ngôi vô địch ASEAN Cup 2024, nhưng riêng hàng công HLV Kim Sang Sik có vẻ như đã tìm được công thức để các chân sút ghi bàn.',
  'content': 'Nỗi ám ảnh từ quá khứ…\nKhông phải cho tới khi HLV Kim Sang Sik nắm quyền, bài toán tấn công hay chất lượng của các tiền đạo luôn là cơn đau đầu rất lớn dành cho ông Park Hang Seo đến HLV Troussier.\nThậm chí, thời điểm HLV Troussier dẫn dắt, mọi thứ còn tệ hơn rất nhiều khi các chân sút thường xuyên bỏ lỡ những cơ hội ngon ăn hoặc bế tắc trong việc tiếp cận khung thành đối thủ.\n[Tuyển Việt Nam thường xuyên gặp vấn đề ở khâu ghi bàn. Ảnh: SN]\nThậm chí khi có thể tung ra những pha dứt điểm, các tiền đạo tuyển Việt Nam dưới thời HLV người Pháp cũng cho thấy độ chính xác là vô cùng thấp.\nChính vì thế, ở một số 

In [ ]:
## Lưu lại vào một cái
with open(os.path.join(DATA_FOLDER, "news_final.json"), "w") as fOut:
    json.dump(all_docs, fOut, indent=4, ensure_ascii=False)

# Tiền xử lí dữ liệu và lưu biểu diễn vào cơ sở dữ liệu

In [4]:
data_path = '/content/drive/MyDrive/AI4E/Project_ML NLP/data/news_final.json'
with open(data_path, 'r') as fIn:
  data = json.load(fIn)
len(data)

20522

In [5]:
stop_words = set([
    "bị", "bởi", "cả", "các", "cái", "cần", "càng", "chỉ", "chiếc", "cho", "chứ", "chưa", "chuyện",
    "có", "có_thể", "cứ", "của", "cùng", "cũng", "đã", "đang", "đây", "để", "đến_nỗi", "đều", "điều",
    "do", "đó", "được", "dưới", "gì", "khi", "không", "là", "lại", "lên", "lúc", "mà", "mỗi", "này",
    "nên", "nếu", "ngay", "nhiều", "như", "nhưng", "những", "nơi", "nữa", "phải", "qua", "ra", "rằng",
    "rằng", "rất", "rồi", "sau", "sẽ", "so", "sự", "tại", "theo", "thì", "trên", "trước", "từ", "từng",
    "và", "vẫn", "vào", "vậy", "vì", "việc", "với", "vừa"
])
def preprocess(text):
  # Chuyển thành chữ thường
  text = text.lower()
  # Loại bỏ dấu câu
  text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)
  # Loại bỏ khoảng trắng thừa
  text = re.sub(r'\s+', ' ', text).strip()
  tokens = text.split()
  # Loại bỏ stop
  tokens = [token for token in tokens if token not in stop_words]
  text = " ".join(tokens)
  return text

In [6]:
texts = []
for article in data:
  texts.append(article['title'] + article['content'])
preprocessed_texts = []
for text in tqdm(texts, desc="Preprocessing texts"):
  preprocessed_text = preprocess(text)
  preprocessed_texts.append(preprocessed_text)

Preprocessing texts: 100%|██████████| 20522/20522 [00:31<00:00, 661.72it/s]


In [7]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(preprocessed_texts)
vocabulary = vectorizer.vocabulary_

In [8]:
tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

In [9]:
tfidf_df_path = '/content/drive/MyDrive/AI4E/Project_ML NLP/data/tfidf_df.csv'
tfidf_df.to_csv('tfidf_df_path', index=False, encoding='utf-8')

In [12]:
with open('/content/drive/MyDrive/AI4E/Project_ML NLP/data/vocabulary.tex', 'w') as f:
    json.dump(vocabulary, f, indent=4, ensure_ascii=False)